### Computational Companion to: "Connected Claw-Free Graphs whose Second Largest Eigenvalue Does Not Exceed 1"

**Authors**: Chaochui Chen, Haiying Shan, Muhuo Liu, Zoran Stanić  
**Software**: SageMath 10.5+  
**Date**: February 2025


---

 #### This code implements calculations for the paper:"Connected claw - free graphs whose second largest eigenvalue does not exceed 1"

### Variable Correspondence Table

| Code Variable      | Paper Notation                          |
|--------------------|-----------------------------------------|
| `G1`               | $\mathcal{G}_1$                         |
| `G2`               | $\mathcal{G}_2$                         |
| `G3`               | $\mathcal{G}_3$                         |
| `H0_minus`         | $H_0^{-}$                               |
| `H0_plus`          | $H_0^{+}$                               |
| `H_plus`           | $H^+$                                   |
| `classified_graphs[i]` | $H_i^{+}$ (i = 1, 2, 3, 4, 5, 6)    |
| `G3_0`             | $\bigcup_{H_1\in \mathcal{H}_{0}^{-}}\mathcal{M}(\mathcal{G}(H_1))$ |

# Basic Function Definitions

In [1]:
# Use ~G to denote the complement of graph G
Graph.__invert__ = Graph.complement

def P(n):
    """
    Returns a path graph with n vertices.
    
    Parameters:
    n (int): Number of vertices in the path graph
    
    Returns:
    Graph: The path graph P_n
    """
    return graphs.PathGraph(n)

def C(n):
    """
    Returns a cycle graph with n vertices.
    
    Parameters:
    n (int): Number of vertices in the cycle graph
    
    Returns:
    Graph: The cycle graph C_n
    """
    return graphs.CycleGraph(n)

def K(*args):
    """
    Returns a complete graph or complete bipartite graph.
    
    Parameters:
    *args: Variable length argument list
        - If one argument: n (int) - number of vertices for complete graph K_n
        - If two arguments: a, b (int) - sizes of bipartition for complete bipartite graph K_{a,b}
    
    Returns:
    Graph: The complete graph K_n or complete bipartite graph K_{a,b}
    """
    if len(args) == 1:
        return graphs.CompleteGraph(args[0])
    elif len(args) == 2:
        return graphs.CompleteBipartiteGraph(args[0], args[1])

def U(List,n):
    """
    Constructs a unicyclic graph U^{List}_{n} with specified pendant vertices attached to cycle vertices.
    
    Parameters:
    List (list): List of integers where List[i] specifies number of pendant vertices to attach to vertex i of the cycle
    n (int): Length of the cycle in the unicyclic graph
    
    Returns:
    Graph: The constructed unicyclic graph U^{List}_{n}
    
    Notes:
    - If len(List) < n, List is padded with zeros
    - The graph consists of an n-cycle with pendant vertices attached according to List
    """
    List = List + [0] * (n - len(List)) if len(List) < n else List
    G = C(n)
    for i,x in enumerate(List):
        if x:
            G.add_edges([(i,j) for j in range(G.order(),G.order()+x)])
    return Graph(G.edges())

def T(a,b,c):
    """
    Returns the tree T_{a,b;c}.
    
    Parameters:
    a (int): Number of leaves attached to the first vertex of the path
    b (int): Number of leaves attached to the last vertex of the path
    c (int): Length of the central path
    
    Returns:
    Graph: The tree T_{a,b;c}
    """
    G = P(c+1)
    G.add_edges([(0,j) for j in range(c+1,c+1+a)] + [(c,j) for j in range(c+1+a,c+1+a+b)])
    return Graph(G.edges())

def B(a,b):
    """
    Returns the graph B_{a,b}.
    
    Parameters:
    a (int): Number of vertices connected to both v1 and v2
    b (int): Number of vertices connected to both v1 and v4
    
    Returns:
    Graph: The graph B_{a,b}
    """
    G = C(4)  
    G.add_vertices(range(4, 4 + a + b))  
    
    for i in range(4, 4 + a):
        G.add_edge(0, i)
        G.add_edge(1, i)
    
    for i in range(4 + a, 4 + a + b):
        G.add_edge(0, i)
        G.add_edge(3, i)
    
    return Graph(G.edges())

def B_minus(a,b):
    """
    Returns the graph B_{a,b}^{-} by removing vertex v3 from B_{a,b}.
    
    Parameters:
    a (int): Number of vertices connected to both v1 and v2
    b (int): Number of vertices connected to both v1 and v4
    
    Returns:
    Graph: The graph B_{a,b}^{-}
    """
    G = B(a,b)
    G.delete_vertex(3)
    return G.Relabel()

def is_eig_2_not_exceeding_1(G):
    """
    Determines if the second largest eigenvalue of a graph G does not exceed 1.
    
    Parameters:
    G: A graph
    
    Returns:
    bool: True if the second largest eigenvalue ≤ 1, False otherwise
    """
    f = G.charpoly('x')
    return f.number_of_roots_in_interval(1, Infinity) - (f(1) == 0) <= 1

def is_eig_min_at_least_neg_2(G):
    """
    Determines if the smallest eigenvalue of a graph G is at least -2.
    
    Parameters:
    G: A graph
    
    Returns:
    bool: True if the smallest eigenvalue ≥ -2, False otherwise
    """
    f=G.charpoly('x')    
    return f.number_of_roots_in_interval(-Infinity, -2) - (f(-2) == 0) == 0

def graph_unique(graphs_list, index_only=False):
    """
    Returns a list of non-isomorphic graphs from the input list.
    
    Parameters:
    graphs_list (list): A list of graphs
    index_only (bool): If True, returns indices of unique graphs instead of the graphs themselves
    
    Returns:
    list: A list of non-isomorphic graphs (or their indices) from the input list
    """
    unique_graphs_dict = {}

    for i, G in enumerate(graphs_list):
        Gstr = G.canonical_label().graph6_string()
        unique_graphs_dict.setdefault(Gstr, []).append(i)
    if index_only:
        return [indices[0] for indices in unique_graphs_dict.values()]
    return [graphs_list[indices[0]] for indices in unique_graphs_dict.values()]

def classify_connected_induced_subgraphs_by_smallest_eigenvalue_less_than_negative_2_or_not(G):
    """
    Classifies all connected induced subgraphs of G based on their smallest eigenvalue.
    
    Parameters:
    G: A graph
    
    Returns:
    list: [L1,L2] where:
        L1: List of connected induced subgraphs with smallest eigenvalue < -2
        L2: List of connected induced subgraphs with smallest eigenvalue ≥ -2
    Note: Each list contains non-isomorphic graphs only
    """
    if is_eig_min_at_least_neg_2(G):
        return [[], list(G.connected_subgraph_iterator())]
    
    all_subgraphs = graph_unique(list(G.connected_subgraph_iterator()))
    
    L1 = [G0.Relabel() for G0 in all_subgraphs if not is_eig_min_at_least_neg_2(G0)]
    L2 = [G0.Relabel() for G0 in all_subgraphs if is_eig_min_at_least_neg_2(G0)]
    
    return [L1, L2]

def minimal_subgraphs(Glist):
    """
    Returns all minimal graphs from a list of graphs.
    
    Parameters:
    Glist (list): A list of graphs
    
    Returns:
    list: A list of minimal graphs from Glist, where a graph G is minimal if no other graph in Glist 
          is a proper induced subgraph of G
    """
    return [
        G for i, G in enumerate(Glist)
        if all(
            not G0.is_subgraph(G, induced=True, up_to_isomorphism=True)
            for G0 in Glist
            if G0.order() < G.order() and all(G0.degree_sequence()[k] <= G.degree_sequence()[k] for k in range(G0.order()))
        )
    ]

def maximal_subgraphs(Glist, index_only=False):
    """
    Returns all maximal graphs from a list of non-isomorphic graphs.
    
    Parameters:
    Glist (list): A list of graphs
    index_only (bool): If True, returns indices of maximal graphs instead of the graphs themselves
    
    Returns:
    list: A list of maximal graphs from Glist (or their indices), where a graph G is maximal if it is not 
          a proper induced subgraph of any other graph in Glist
    """
    maximal_graphs_index = [
        i for i, G in enumerate(Glist)
        if all(
            not G.is_subgraph(G0, induced=True, up_to_isomorphism=True)
            for G0 in Glist
            if G0.order() > G.order() and all(G0.degree_sequence()[k] >= G.degree_sequence()[k] for k in range(G.order()))
        )
    ]

    return maximal_graphs_index if index_only else [Glist[i] for i in maximal_graphs_index]
    
def maximal_independent_number_for_fixed_VG_minus_H1(G):
    """
    Returns the maximum size of isolated vertices such that the second largest eigenvalue of the complement 
    of G union these isolated vertices does not exceed 1.
    
    Parameters:
    G: A graph
    
    Returns:
    int: Maximum number of isolated vertices that can be added while keeping λ2 ≤ 1, or -1 if λ2 of ~G > 1
    """
    if not is_eig_2_not_exceeding_1(~G):
        return -1
    k = 0
    while is_eig_2_not_exceeding_1(~(G + (k+1)*K(1))):
        k += 1
    return k

# Specified graphs shown in Figures 1-3

In [2]:
D1=Graph('LhCGOO_GC?O?_?')
D2=Graph('LhCGOOG_@?C?_?')
D3=Graph('LhCGOQ?G@?@?A?')
D4=Graph('LhC`?OGA?O@?A?')
D5=Graph('GlIHCC')
D6=Graph('LlIKCA?_C?O?G?')
D7=Graph('IlIHCA?G?')
D8=Graph('Ihd@CCMDO')
D9=Graph('Khe?GCI?_C?O')
D10=Graph('Khc_S?B?G@?C')
D11=Graph('Khc_S?B?G@C?')
D12=Graph('Mhe?GC@?G@GOC?C??')
D13=Graph('Khc_c?@?G@?C')
D14=Graph('Lhc_c?@?G?_@?A')
D15=Graph('Ihe?H?BCG')
D16=Graph('Khe?H?A?GAa@')
D17=Graph('IhcGGcAO?')
D18=Graph('IhcGGcA?G')
D19=Graph('IhcGa?@?G')
D20=Graph('Ghc_Gc')
D21=Graph('KhCGOO__C?O?')
D22=Graph('Khe?GC@?ICA?')
D23=Graph('Khc_c?@?G?_@')
D24=Graph('Jk_GGC@?QG?')
D25=Graph('K^{rxcqnwsdQ')
D26=Graph('J^lwww[J?w?')
D27=Graph('H{L[`TF')
D28=Graph('KFCO_ao^w__E')
D29=Graph('JXG`F@~?XB?')
D30=Graph('KX^hXWkB?W@_')
D31=Graph('KnTRwf^CiDAS')

In [3]:
Z1=Graph('D][')
Z2=Graph('Kh_OOCC?_?_C')
Z3=Graph('ErGW')
Z4=Graph('Er?W')
Z5=Graph('FiCOG')
Z6=Graph('GUC_Og')
Z7=Graph('JdO_o?@?GG_')
Z8=Graph('KdO_oGC@?G?_')
Z9=Graph('I`d@@?K?O')
Z10=Graph('I`d@?CK?_')
Z11=Graph('KdH??OS??E_@')
Z12=Graph('J`d@?W?@GC?')
Z13=Graph('KdO_o?H@?G?_')
Z14=Graph('HkC_OGP')
Z15=Graph('JqGG_OC?_A_')
Z16=Graph('K^]www[J?wB_')
Z17=Graph('ErCo')
Z18=Graph('ErOw')
Z19=Graph('LXxxXWkJ?W@_B?')
Z20=Graph('L?{GOOGUD~`@?E')
Z21=Graph('KFCO_ao^w@aE')
Z22=Graph('KXxxXWKB?W@_')
Z23=Graph('L^Nx`_o^|~abHE')
Z24=Graph('LnTRycQC_cZ~O`')
Z25=Graph('KnTwggSvwgga')

In [4]:
W1=Graph('FkT@g')
W2=Graph('DzK')
W3=Graph('ExdW')
W4=Graph('EpGG')
W5=Graph('GiCGOC')
W6=Graph('HkCGOC@')
W7=Graph('IpGG?GB?G')
W8=Graph('Ih?WOO@?g')
W9=Graph('IKOG_O@?g')

## The computation for the forbidden graphs in Lemma 3.5

In [5]:
# type (i) (except for Z_1 and K_{2,3}) in Lemma 3.5
forbidden_graphs_1 = (
    [Z2, Z3, Z4, Z5, Z6, Z7, Z8, Z9, Z10, Z11, Z12, Z13, Z14, Z15, Z16, 
     Z17, Z18, Z19, Z20, Z21, Z22, Z23, Z24, Z25] 
    + [T(7, 3, 2), T(6, 6, 2), T(7, 1, 3), T(3, 2, 1), T(3, 2, 3), 
       T(4, 1, 5), T(3, 2, 5), U([7, 1, 1], 3)]
    + [U([1], i) for i in [6, 8, 10, 11]]
    + [U([1, 1], 4), U([5, 0, 4], 4), U([7, 0, 3], 4), U([3], 7), 
       U([1, 1], 7), U([2], 9), U([1, 1], 9), U([1, 0, 1], 9), 
       U([1, 0, 0, 1], 9)]
    + [B_minus(3, 7), B_minus(4, 5), B_minus(5, 4), B_minus(7, 3)]
)

# type (ii) in Lemma 3.5
forbidden_graphs_2 = (
    [G + K(1) for G in [U([1], 4), U([5], 5), T(9, 1, 1), K(1, 17) + K(2)]]
    + [K(1, 17) + 3 * K(1)]
)

# type (iii) in Lemma 3.5
forbidden_graphs_3 = [
    G + 7 * K(1) for G in [K(1, 5), T(3, 1, 1), W5, W6, W7, W8, W9]
]

forbidden_graphs = forbidden_graphs_1 + forbidden_graphs_2 + forbidden_graphs_3

flag = True
for Z in forbidden_graphs:
    if is_eig_2_not_exceeding_1(~Z):
        flag = False
if flag == True:
    print('The second largest eigenvalue of every graph in the list is greater than 1.')

The second largest eigenvalue of every graph in the list is greater than 1.


## The computation for $\mathcal{G}_1\cup \mathcal{G}_2$ in Theorem 4.3

In [6]:
G1 = [D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16] + [U([4,0,4],4),U([6,0,3],4),U([6,0,1,1],5),U([2,0,2],5),U([2,0,2],7),U([1,0,2,0,1],7),U([1,0,0,0,1],9)]
G2 = [D25,D26,D27,D28,D29,D30,D31] + [U([2,2,2],3),U([6,1,1],3),B(6,3),B(4,4)]

flag = True
for G in G1 + G2:
    if not is_eig_2_not_exceeding_1(~G):
        flag = False
if flag == True:
    print('The second largest eigenvalue of every graph in the list is not exceeding 1.')

The second largest eigenvalue of every graph in the list is not exceeding 1.


## The computation for $\bigcup_{i=2}^{8} \mathcal{F}_i$ in Theorem 4.3

In [7]:
A2_dict = {
    2: T(0, 2, 2).adjacency_matrix(),
    3: U([0, 0, 2], 4).adjacency_matrix(),
    4: (K(1) + C(4)).adjacency_matrix(),
    5: W1.adjacency_matrix(),
    6: W2.adjacency_matrix(),
    7: W3.adjacency_matrix(),
    8: B(0, 2).adjacency_matrix()
}

beta_dict = {
    2: matrix([[1,0,0,0,0]]).transpose(),
    3: matrix([[1,0,0,0,0,0]]).transpose(),
    4: matrix([[1,0,0,0,0]]).transpose(),
    5: matrix([[0,1,0,0,0,0,0]]).transpose(),
    6: matrix([[0,1,0,1,0]]).transpose(),
    7: matrix([[1,0,0,0,1,0]]).transpose(),
    8: matrix([[1,1,0,0,0,0]]).transpose()
}

In [8]:
flag = True
for i in range(2, 9):
    A2=A2_dict[i]
    beta=beta_dict[i]
    j = matrix(A2.nrows()*[1]).transpose()
    matrix_result = A2 - j*beta.transpose() - beta*j.transpose() + beta * beta.transpose()
    if not is_eig_min_at_least_neg_2(matrix_result):
        flag = False
if flag == True:
    print('The smallest eigenvalue of every matrix_result is at least -2.')

The smallest eigenvalue of every matrix_result is at least -2.


## The computation for Fact 1 in Theorem 5.1

In [9]:
# Generate all trees of order not exceeding 14
all_trees = []
for n in range(1, 15):  # From 1 to 14
    trees_of_order_n = list(graphs.trees(n))  # Generate trees of order n
    all_trees.extend(trees_of_order_n)  # Add to the total list

print(len(all_trees))
# Filter trees whose second largest eigenvalue of the complement does not exceed 1
valid_trees = []
for tree in all_trees:
    if not is_eig_2_not_exceeding_1(~tree):
        continue
    valid_trees.append(tree)

# Generate all maximal trees among them
maximal_trees = maximal_subgraphs(valid_trees)

5447


In [10]:
GL = G1 + [T(2,2,8),T(1,1,11),T(2,1,10),T(2,2,9),K(1,13),T(11,1,1),T(10,1,2),T(9,2,2)]

flag = True
for Tree in maximal_trees:
    # Check if Tree is an induced subgraph of some graph in GL
    is_induced_subgraph = False
    for G in GL:
        if Tree.order()>G.order() or any(Tree.degree_sequence()[i]>G.degree_sequence()[i] for i in range(Tree.order())):
            continue        
        if Tree.is_subgraph(G, induced=True, up_to_isomorphism=True):
            is_induced_subgraph = True
            break
    if not is_induced_subgraph:
        flag = False
if flag == True:
    print('Every maximal tree is an induced subgraph of some graph in GL.')

Every maximal tree is an induced subgraph of some graph in GL.


# Generate $H_0^+$ and $H_0^-$ as defined preceding Lemma 6.4

In [11]:
H0_minus = []
H0_plus = []
for G in G1 + [T(8,2,2), K(1,16)]:
    minus_graphs, plus_graphs = classify_connected_induced_subgraphs_by_smallest_eigenvalue_less_than_negative_2_or_not(G)
    H0_minus.extend(minus_graphs)
    H0_plus.extend(plus_graphs)
H0_minus = graph_unique(H0_minus)
H0_plus = graph_unique(H0_plus)
[len(H0_minus),len(H0_plus)]

[198, 64]

## Generate $H^+$ as defined following Lemma 6.6

In [12]:
minimal_H0_minus = minimal_subgraphs(H0_minus)
maximal_H0_plus = maximal_subgraphs(H0_plus)
maximal_H_plus = [K(1,4), D8, D15, D17, D18, D19, T(2,1,7)] + [C(i) for i in range(4,14)] + [T(2,2,i) for i in range(1,6)]
H_plus = graph_unique(sum([list(G.connected_subgraph_iterator()) for G in maximal_H_plus], []))
len(H_plus)

67

# The three-step algorithm following Lemma 6.7

In [13]:
from collections import Counter
from itertools import product, chain

def classify_graphs_by_independence_number(Glist):
    """Classify graphs by their independence number into categories i=1 to 6"""
    classified_graphs = {i: [] for i in range (1,7)}
    for G in Glist:
        i = len(G.independent_set())
        classified_graphs[i].append(G)
    return classified_graphs

def index2graph(classified_graphs, Index):
    """Reconstruct graph from index representation
    Args:
        classified_graphs: dict storing graphs categorized by independence number
        Index: list of tuples (category_idx, graph_idx) specifying which graphs to combine
    Returns:
        Disjoint union of specified graphs
    """
    H_2=graphs.EmptyGraph()
    # Each entry in Index is a tuple (a,b) representing classified_graphs[a][b]
    for L in Index:
        H_2=H_2+classified_graphs[L[0]][L[1]]
    return H_2

def generate_possible_H_light(classified_graphs):
    """Generate possible combinations for G-H_1 components
    Returns:
        Nested dictionary structure:
        - Key level 1: independence number j (1-6)
        - Key level 2: number of components i
        - Value: list of index combinations for i components from j's graph list
    """
    return {
        j: [
            [[(j, M1) for M1 in M]  # Create index tuples for each component
            for M in unordered_tuples(range(len(classified_graphs[j])), i)]
            # Limit maximum components based on independence number
            for i in range(0, 7 if j == 1 else 4 if j == 2 else 3 if j == 3 else 2)            
        ]
        for j in range(1, 7)
    }

def generate_G3_0(classified_graphs, H0_minus):
    """Generate index list for G3_0 candidates
    Args:
        classified_graphs: pre-classified graph collections
        H0_minus: list of graphs with λ_min < -2
    """
    G3_0 = []
    possible_H_light = generate_possible_H_light(classified_graphs)

    # Iterate through all H_1 candidates in H0_minus
    for a, H_1 in enumerate(H0_minus):
        k = maximal_independent_number_for_fixed_VG_minus_H1(H_1)
        valid_indices = []

        # Generate all valid component combinations
        for i in range(1, k + 1):
            for L in Partitions(i):  # Integer partitions of i
                L_count = Counter(L)
                # Get possible component combinations for current partition
                possible_l = [possible_H_light[l][c] for l, c in L_count.items()]
                # Cartesian product of all component possibilities, corresponding to all possible graphs in Lambda(H_1)
                for L1 in product(*possible_l):
                    flatten_L1 = list(chain.from_iterable(L1))
                    H_light = index2graph(classified_graphs, flatten_L1)
                    # Check eigenvalue condition for complement graph
                    if is_eig_2_not_exceeding_1(~(H_1 + H_light)):
                        valid_indices.append(flatten_L1)

        # Keep only maximal valid configurations
        valid_graphs = [index2graph(classified_graphs, L1) for L1 in valid_indices] # corresponds to \mathcal{G}(H_1)
        maximal_valid_indices = [valid_indices[i] for i in maximal_subgraphs(valid_graphs, index_only=True)] # the indices of the maximal graphs in \mathcal{G}(H_1)

        # Combine H_1 index with valid H_light indices
        G3_0.extend([[(0, a)] + H_light_index for H_light_index in maximal_valid_indices])

    return G3_0

# Main logic flow:
# 1. Classify H_plus graphs by independence number (1-6)
# 2. Store H0_minus in classified_graphs[0]
# 3. Generate candidate indices for G3_0
classified_graphs = classify_graphs_by_independence_number(H_plus)
classified_graphs[0] = H0_minus
G3_0 = generate_G3_0(classified_graphs, H0_minus)
[len(classified_graphs[i]) for i in range(0,7)],len(G3_0)

([198, 2, 4, 10, 24, 17, 10], 177)

In [14]:
from sage.graphs.connectivity import connected_components_subgraphs

def is_star(G):
    # Determines whether graph G is a star graph
    return max(G.degree())==G.size()
    
def is_4_cycle(G):
    # Determines whether graph G is a 4-cycle
    return G.is_cycle() and G.order()==4

def is_the_union_of_a_star_and_a_4_cycle(G):
    # Determines whether graph G is a disjoint union of a star graph and a 4-cycle
    from sage.graphs.connectivity import connected_components_subgraphs
    all_connected_components = connected_components_subgraphs(G)
    if len(all_connected_components) == 2:
        C1, C2 = all_connected_components
        return (is_star(C1) and is_4_cycle(C2)) or (is_star(C2) and is_4_cycle(C1))
    return False


In [15]:
from sage.combinat.combination import Combinations
# Filter out graphs contained in 𝒢₁ or ℱ₄  
G3_1 = []  # Stores indices after removing graphs contained in 𝒢₁ or ℱ₄ 
for ind in G3_0:
    G = index2graph(classified_graphs, ind)
    if not is_the_union_of_a_star_and_a_4_cycle(G) and not any(G.is_subgraph(G0, induced=True, up_to_isomorphism=True) for G0 in G1):
        G3_1.append(ind)

Theoretically, we could directly compute maximal graphs from G3_1 to obtain G3_index. However, 
this approach fails due to issues with the 'is_subgraph' function when handling graphs containing 
high-order star graphs as induced subgraphs (e.g., 'T(10,2,1).is_subgraph(K(1,13), induced=True, 
up_to_isomorphism=True)' fails). Therefore, we implemented optimizations:

<div style="line-height: 1.2;">
    
1. Filter G3_1 to remove graphs where both $H_{\text{heavy}}$ and $H_{\text{light}}$ are proper 
   induced subgraphs of another graph's $H_{\text{heavy}}$ and $H_{\text{light}}$ respectively, 
   resulting in G3_2.

2. Note that since $\lambda_{\min}(K_{1,5}) < -2$, star graphs of order ≥6 can only appear in 
   $H_{\text{heavy}}$.

3. Classify G3_2 into "star_graphs" and "no_star_graphs" based on whether $H_{\text{heavy}}$ 
   is a star graph.

4. Compute maximal graphs from no_star_graphs (their indices), verify that no star_graphs are 
   induced subgraphs of these maximal graphs, and finally combine them into G3_index while 
   converting to graph list G3.
</div>

In [16]:
# Extract unique heavy component indices that appear in G3_1
heavy_indices = list({L[0][1] for L in G3_1})  # Using set to ensure uniqueness

# Build subgraph relationship map between heavy components
subgraph_map = {i: [] for i in heavy_indices}
for i, j in Combinations(heavy_indices, 2):
    G_i, G_j = H0_minus[i], H0_minus[j]
    if G_j.is_subgraph(G_i, induced=True, up_to_isomorphism=True):
        subgraph_map[j].append(i)
    if G_i.is_subgraph(G_j, induced=True, up_to_isomorphism=True):
        subgraph_map[i].append(j)

# Filter graphs where both components are maximal
G3_2 = [
    ind for ind in G3_1
    if not any(
        other_ind[0][1] in subgraph_map[ind[0][1]] and 
        index2graph(classified_graphs, ind[1:]).is_subgraph(
            index2graph(classified_graphs, other_ind[1:]), 
            induced=True, up_to_isomorphism=True
        )
        for other_ind in G3_1
    )
]

# Handle star components and finalize maximal graphs
star_candidates = [ind for ind in G3_2 if is_star(classified_graphs[ind[0][0]][ind[0][1]])]  # Star graphs ≥6 are automatically maximal
no_star_candidates = [ind for ind in G3_2 if not is_star(classified_graphs[ind[0][0]][ind[0][1]])]

# Add maximal graphs with no star components
star_graphs = [index2graph(classified_graphs, ind) for ind in star_candidates]
no_star_graphs = [index2graph(classified_graphs, ind) for ind in no_star_candidates]
maximal_indices = maximal_subgraphs(no_star_graphs, index_only=True)

# Check if any star_graphs are induced subgraphs of no_star_graphs
for Gi in star_graphs:
    for Gj in no_star_graphs:
        if Gi.is_subgraph(Gj, induced=True, up_to_isomorphism=True):
            print(Gi, Gj)
            break

# Merge star_candidates and no_star_candidates
G3_index = star_candidates + [no_star_candidates[i] for i in maximal_indices]
# Final graph list
G3 = [index2graph(classified_graphs, ind) for ind in G3_index]
len(G3)

74

In [17]:
# Extract unique valid indices from G3_index
G3_valid_index = list(set([ind1 for ind in G3_index for ind1 in ind]))

# Manually map these indices to their corresponding graph types in a dictionary
G3_components_dict = {
    (4, 12): 'D_{20}',
    (0, 57): 'K_{1,7}',
    (2, 2): 'C_{5}',
    (1, 0): 'K_{1}',
    (0, 8): 'D_{21}',
    (0, 127): 'D_{11}',
    (0, 191): 'K_{1,10}',
    (0, 197): 'K_{1,16}',
    (0, 38): 'K_{1,5}',
    (0, 157): 'D_{24}',
    (3, 9): 'C_{7}',
    (5, 6): 'T_{2,2;3}',
    (3, 6): 'C_{6}',
    (4, 17): 'C_{8}',
    (0, 117): 'U_{5}^{2,0,1,1}',
    (5, 15): 'C_{11}',
    (0, 187): 'U_{7}^{1,0,1,0,1}',
    (4, 23): 'C_{9}',
    (2, 1): 'P_4',
    (0, 193): 'K_{1,12}',
    (0, 22): 'T_{3,1;2}',
    (0, 104): 'K_{1,8}',
    (0, 107): 'U_{5}^{2,0,2}',
    (4, 4): 'D_{8}',
    (0, 165): 'D_{23}',
    (0, 171): 'D_{16}',
    (0, 177): 'T_{4,4;2}',
    (0, 49): 'K_{1,6}',
    (5, 14): 'C_{10}',
    (1, 1): 'K_{2}',
    (2, 3): 'C_{4}',
    (0, 137): 'D_{22}',
    (0, 195): 'K_{1,14}',
    (0, 21): 'T_{3,1;1}',
    (6, 6): 'C_{12}',
    (0, 24): 'T_{3,3;2}',
}
union_symbol = r' \cup '
# Represent each graph in G3 as a disjoint union of the corresponding graphs in G3_components_dict
for i, G in enumerate(G3):
    G.name(f"${union_symbol.join(G3_components_dict[k] for k in G3_index[i])}$")
G3_rows = [G3[i:i+7] for i in range(0, len(G3), 7)]
print('G3 consists of the following graphs:')
table([[G.name() for G in G3_row] for G3_row in G3_rows])

G3 consists of the following graphs:


$K_{1,5} \cup T_{2,2;3}$                       $K_{1,5} \cup C_{11}$                          $K_{1,5} \cup C_{9} \cup K_{2}$         $K_{1,5} \cup C_{6} \cup C_{5}$
  $K_{1,5} \cup C_{5} \cup C_{4} \cup K_{2}$     $K_{1,5} \cup C_{12}$                          $K_{1,5} \cup C_{10} \cup K_{1}$        $K_{1,5} \cup D_{8} \cup C_{4}$
  $K_{1,5} \cup C_{4} \cup C_{4} \cup C_{4}$     $K_{1,6} \cup C_{7}$                           $K_{1,6} \cup C_{5} \cup K_{2}$         $K_{1,6} \cup D_{8}$
  $K_{1,7} \cup C_{4} \cup K_{2}$                $K_{1,8} \cup C_{6}$                           $K_{1,8} \cup C_{4} \cup K_{1}$         $K_{1,10} \cup K_{2} \cup K_{2}$
  $D_{21} \cup K_{1}$                            $T_{3,1;1} \cup T_{2,2;3}$                     $T_{3,1;1} \cup C_{11}$                 $T_{3,1;1} \cup C_{9} \cup K_{2}$
  $T_{3,1;1} \cup C_{5} \cup C_{5} \cup K_{1}$   $T_{3,1;1} \cup C_{5} \cup C_{4} \cup K_{2}$   $T_{3,1;1} \cup C_{12}$                 $T_{3,1;1} \cup C_{10} \cup K_{1}$
  $T_{3,1;1} \cup C_{6} \cup C_{6}$              $T_{3,1;1} \cup C_{4} \cup C_{4} \cup C_{4}$   $T_{3,1;2} \cup C_{9}$                  $T_{3,1;2} \cup C_{7} \cup K_{2}$
  $T_{3,1;2} \cup C_{6} \cup C_{4}$              $T_{3,3;2} \cup C_{5}$                         $T_{3,3;2} \cup C_{6}$                  $T_{3,3;2} \cup C_{4} \cup K_{1}$
  $U_{5}^{2,0,2} \cup D_{20}$                    $U_{5}^{2,0,2} \cup C_{8}$                     $U_{5}^{2,0,2} \cup C_{4} \cup C_{4}$   $U_{5}^{2,0,1,1} \cup C_{7}$
  $U_{5}^{2,0,1,1} \cup C_{8}$                   $U_{5}^{2,0,1,1} \cup C_{4} \cup C_{4}$        $D_{11} \cup C_{4}$                     $D_{22} \cup K_{1}$
  $T_{4,4;2} \cup C_{4}$                         $U_{7}^{1,0,1,0,1} \cup C_{5}$                 $U_{7}^{1,0,1,0,1} \cup C_{6}$          $U_{7}^{1,0,1,0,1} \cup C_{4} \cup K_{1}$

## The computation for $\mathcal{G}_3$ in Theorem 4.3

In [18]:
# Filter graphs in G3 where the complement graph's second eigenvalue exceeds 1
flag = True
for G in G3:
    if not is_eig_2_not_exceeding_1(~G):
        flag = False
if flag == True:
    print('The second largest eigenvalue of every graph in the list is not exceeding 1.')

The second largest eigenvalue of every graph in the list is not exceeding 1.
